In [310]:
import os
import numpy as np
import pandas as pd
from nilearn.connectome import ConnectivityMeasure
from sklearn.preprocessing import StandardScaler

In [311]:
# Tian 1:10, 17:26
subcortical_index = list(range(0,10)) + list(range(16,26))
# Schaefer: 
# lh-mPFC: 199:205
# rh-mPFC: 464:470
# lh-Ins: 67, 108:111, 126:128
# rh-Ins: 319, 361:364, 383:386
## ACC: 390
# Glasser
cortical_roi = ['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins']
lh_dlPFC_index = [205, 246, 247, 249, 250, 252, 262, 263, 264, 265, 266, 276, 277]
rh_dlPFC_index = [25, 66, 67, 69, 70, 72, 82, 83, 84, 85, 86, 96, 97]
lh_mPFC_index = [236, 237, 238, 239, 240, 241, 242, 243, 244, 248, 267, 343, 344, 345, 358, 359]
rh_mPFC_index = [56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 87, 163, 164, 165, 178, 179]
lh_PCC_index = [193, 194, 206, 209, 210, 211, 212, 213, 214, 300, 321, 340, 341]
rh_PCC_index = [13, 14, 26, 29, 30, 31, 32, 33, 34, 120, 141, 160, 161]
lh_Ins_index = [285, 287, 288, 289, 290, 291, 293, 294, 346, 347, 348, 357]
rh_Ins_index = [105, 107, 108, 109, 110, 111, 113, 114, 166, 167, 168, 177]
dic_cortical_roi = {
    'lh_dlPFC': lh_dlPFC_index,
    'rh_dlPFC': rh_dlPFC_index,
    'lh_mPFC': lh_mPFC_index,
    'rh_mPFC': rh_mPFC_index,
    'lh_PCC': lh_PCC_index,
    'rh_PCC': rh_PCC_index,
    'lh_Ins': lh_Ins_index,
    'rh_Ins': rh_Ins_index
}

In [312]:
user_dir = '/Users/xiaoqianxiao'
projectName = 'UKB'
data_dir = os.path.join(user_dir, projectName, "data")
derivatives_dir = os.path.join(data_dir, 'derivatives')
fMRIinfo_file_path = os.path.join(data_dir, 'current_anxiety_data_set.csv')
df_fMRIinfo = pd.read_csv(fMRIinfo_file_path)
participant_file_path = os.path.join(data_dir, 'participants_fMRI.csv')
df_participants = pd.read_csv(participant_file_path)
#subject_IDs = participants_df['eid']
subject_IDs = df_fMRIinfo['eid'].unique()
#for each subject:
#subject_ID = subject_IDs[3]
session_ID = 2
#load timeseries
#session_ID in range(2,4):
# Initialize lists to hold the data
X = []  # To hold the time series data
subject_id_list_ori = []  # To hold the subject IDs

# Loop through the list of subject IDs
for subject_ID_ori in subject_IDs:
    df_sub_session = pd.DataFrame()  # Initialize an empty DataFrame for each subject
    cortical_file_name = f"sub-{subject_ID_ori}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, 'timeseries/current_anxiety_data_set', cortical_file_name)
    subcortical_file_name = f"sub-{subject_ID_ori}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, 'timeseries/current_anxiety_data_set',subcortical_file_name)
    if os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path):
        # Load the data file and concatenate to the subject's data
        ## cortical ROIs
        df_cortical_all = pd.read_csv(cortical_file_path, compression='gzip', index_col=0, header=0)
        df_cortical_roi = pd.DataFrame({
            roi: df_cortical_all.iloc[dic_cortical_roi[roi]].mean(axis=0)  # Calculate mean time series for each ROI
            for roi in dic_cortical_roi.keys()
        })
        ##subcortical ROIs
        df_subcortical_all = pd.read_csv(subcortical_file_path, compression='gzip', index_col=0, header=0)
        df_subcortical_roi = df_subcortical_all.iloc[subcortical_index]
        # gather all ROIs into one dataframe
        df_roi = pd.concat([df_cortical_roi, df_subcortical_roi.transpose()], axis=1)
        # Append combined data and subject ID to respective lists
        X.append(df_roi.values)  # Store the time-series matrix
        subject_id_list_ori.append(subject_ID_ori)
    else:
        print(f"Missing files for subject {subject_ID_ori}, session {session_ID}.")
X_cleaned = []
# Step 1: Handle NaN values by filling them with the mean of each feature
for i, data in enumerate(X):
    # Skip empty arrays
    if data.size == 0:
        print(f"Subject {i + 1} has an empty array. Skipping.")
        continue

    if np.all(np.isnan(data)):  # Check if all values are NaN
        print(f"Subject {i + 1} has all NaN values. Filling with zeros.")
        data_filled = np.zeros_like(data)
        X_cleaned.append(data_filled)
        continue

    # Fill NaNs column-wise with the mean of each feature
    data_filled = np.copy(data)  # Make a copy of the data to modify
    for j in range(data.shape[1]):  # Iterate over each feature (column)
        if np.isnan(data[:, j]).any():  # If the column contains NaNs
            if np.all(np.isnan(data[:, j])):  # If all values in the column are NaN
                print(f"Column {j} in Subject {i + 1} has all NaN values. Filling with zeros.")
                data_filled[:, j] = np.zeros(data.shape[0])  # Fill the entire column with zeros
            else:
                feature_mean = np.nanmean(data[:, j])  # Compute mean ignoring NaNs
                data_filled[:, j] = np.nan_to_num(data[:, j], nan=feature_mean)  # Replace NaNs with the mean of the column

    X_cleaned.append(data_filled)

# Step 2: Remove constant features
X_filtered = []
subject_id_list_filtered = []
for i, data in enumerate(X_cleaned):
    subject_ID = subject_id_list_ori[i]
    non_constant_features = data[:, data.std(axis=0) != 0]
    if non_constant_features.shape[1] < data.shape[1]:
        print(f"Removed constant features for Subject {i + 1}.")
    if non_constant_features.size == data.size: #only append if non constant features
		#non_constant_features.size > 0:  # Only append if there are remaining features
        X_filtered.append(non_constant_features)
        subject_id_list_filtered.append(subject_ID)

# Step 3: Final check for any remaining NaNs
X_final = []
for i, data in enumerate(X_filtered):
    if np.isnan(data).any():
        print(f"Subject {i + 1} still has NaN values after filtering. Filling remaining NaNs with zeros.")
        data_filled = np.nan_to_num(data, nan=0)  # Fill any remaining NaNs with zeros
        X_final.append(data_filled)
    else:
        X_final.append(data)

# Ensure consistency in the number of features
if not all(data.shape[1] == X_final[0].shape[1] for data in X_final):
    raise ValueError("All subjects must have the same number of features before standardization.")

# Step 3: Standardize data
cad_X_standardized = [StandardScaler().fit_transform(data) for data in X_final]

# Initialize ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
# Fit and transform to compute connectivity matrices
connectivity_matrices = correlation_measure.fit_transform(cad_X_standardized)
# Use numpy to get the upper triangle of each connectivity matrix
num_subjects = connectivity_matrices.shape[0]
num_nodes = connectivity_matrices.shape[1]

# Get upper triangle indices (excluding diagonal)
upper_tri_indices = np.triu_indices(num_nodes, k=1)

# Flatten and store the upper triangle values in the desired shape
cad_upper_triangle_flattened = np.empty((num_subjects, len(upper_tri_indices[0])))

# Extract upper triangle values for each subject
for i in range(num_subjects):
    cad_upper_triangle_flattened[i] = connectivity_matrices[i][upper_tri_indices]

# Check the shape of the result
print(cad_upper_triangle_flattened.shape) 

sample_size = cad_upper_triangle_flattened.shape[1]
df_CAD = df_participants.loc[df_participants['eid'].isin(subject_id_list_filtered)]
#hospital_current_anxiety is the label for classification
## after data clean, end up with 451 [454] CAD and 416[417] controls

X_cad = cad_upper_triangle_flattened  # Feature matrix
y_cad = df_CAD['hospital_current_anxiety']  # Target variable (e.g., symptom scores)

Missing files for subject 1371776, session 2.
Missing files for subject 5260648, session 2.
Missing files for subject 1499481, session 2.
Missing files for subject 5360421, session 2.
Missing files for subject 4048413, session 2.
Missing files for subject 2497476, session 2.
Missing files for subject 2248736, session 2.
Missing files for subject 3301731, session 2.
Missing files for subject 1894259, session 2.
Missing files for subject 2943763, session 2.
Missing files for subject 2687327, session 2.
Missing files for subject 3266434, session 2.
Missing files for subject 1881186, session 2.
Missing files for subject 4042903, session 2.
Missing files for subject 2565815, session 2.
Missing files for subject 2598513, session 2.
Missing files for subject 2660451, session 2.
Missing files for subject 5591491, session 2.
Missing files for subject 3694500, session 2.
Missing files for subject 2639017, session 2.
Missing files for subject 1562628, session 2.
Missing files for subject 3686416,

In [319]:
len(subject_id_list_ori)

118

In [309]:
y_cad_GAD = df_CAD.loc[df_CAD['GAD7_score'] >=0]


In [231]:
user_dir = '/Users/xiaoqianxiao'
projectName = 'UKB'
data_dir = os.path.join(user_dir, projectName, "data")
derivatives_dir = os.path.join(data_dir, 'derivatives')
fMRIinfo_file_path = os.path.join(data_dir, 'past_anxiety_data_set.csv')
df_fMRIinfo = pd.read_csv(fMRIinfo_file_path)
participant_file_path = os.path.join(data_dir, 'participants_fMRI.csv')
df_participants = pd.read_csv(participant_file_path)
#subject_IDs = participants_df['eid']
subject_IDs = df_fMRIinfo['eid'].unique()
#for each subject:
#subject_ID = subject_IDs[3]
session_ID = 2
#load timeseries
#session_ID in range(2,4):
# Initialize lists to hold the data
X = []  # To hold the time series data
subject_id_list_ori = []  # To hold the subject IDs

# Loop through the list of subject IDs
for subject_ID_ori in subject_IDs:
    df_sub_session = pd.DataFrame()  # Initialize an empty DataFrame for each subject
    cortical_file_name = f"sub-{subject_ID_ori}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, 'timeseries/past_anxiety_data_set', cortical_file_name)
    subcortical_file_name = f"sub-{subject_ID_ori}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, 'timeseries/past_anxiety_data_set',subcortical_file_name)
    if os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path):
        # Load the data file and concatenate to the subject's data
        ## cortical ROIs
        df_cortical_all = pd.read_csv(cortical_file_path, compression='gzip', index_col=0, header=0)
        df_cortical_roi = pd.DataFrame({
            roi: df_cortical_all.iloc[dic_cortical_roi[roi]].mean(axis=0)  # Calculate mean time series for each ROI
            for roi in dic_cortical_roi.keys()
        })
        ##subcortical ROIs
        df_subcortical_all = pd.read_csv(subcortical_file_path, compression='gzip', index_col=0, header=0)
        df_subcortical_roi = df_subcortical_all.iloc[subcortical_index]
        # gather all ROIs into one dataframe
        df_roi = pd.concat([df_cortical_roi, df_subcortical_roi.transpose()], axis=1)
        # Append combined data and subject ID to respective lists
        X.append(df_roi.values)  # Store the time-series matrix
        subject_id_list_ori.append(subject_ID_ori)
    else:
        print(f"Missing files for subject {subject_ID_ori}, session {session_ID}.")

X_cleaned = []
# Step 1: Handle NaN values by filling them with the mean of each feature
for i, data in enumerate(X):
    # Skip empty arrays
    if data.size == 0:
        print(f"Subject {i + 1} has an empty array. Skipping.")
        continue

    if np.all(np.isnan(data)):  # Check if all values are NaN
        print(f"Subject {i + 1} has all NaN values. Filling with zeros.")
        data_filled = np.zeros_like(data)
        X_cleaned.append(data_filled)
        continue

    # Fill NaNs column-wise with the mean of each feature
    data_filled = np.copy(data)  # Make a copy of the data to modify
    for j in range(data.shape[1]):  # Iterate over each feature (column)
        if np.isnan(data[:, j]).any():  # If the column contains NaNs
            if np.all(np.isnan(data[:, j])):  # If all values in the column are NaN
                print(f"Column {j} in Subject {i + 1} has all NaN values. Filling with zeros.")
                data_filled[:, j] = np.zeros(data.shape[0])  # Fill the entire column with zeros
            else:
                feature_mean = np.nanmean(data[:, j])  # Compute mean ignoring NaNs
                data_filled[:, j] = np.nan_to_num(data[:, j], nan=feature_mean)  # Replace NaNs with the mean of the column

    X_cleaned.append(data_filled)

# Step 2: Remove constant features
X_filtered = []
subject_id_list_filtered = []
for i, data in enumerate(X_cleaned):
    subject_ID = subject_id_list_ori[i]
    non_constant_features = data[:, data.std(axis=0) != 0]
    if non_constant_features.shape[1] < data.shape[1]:
        print(f"Removed constant features for Subject {i + 1}.")
    if non_constant_features.size == data.size: #only append if non constant features
		#non_constant_features.size > 0:  # Only append if there are remaining features
        X_filtered.append(non_constant_features)
        subject_id_list_filtered.append(subject_ID)

# Step 3: Final check for any remaining NaNs
X_final = []
for i, data in enumerate(X_filtered):
    if np.isnan(data).any():
        print(f"Subject {i + 1} still has NaN values after filtering. Filling remaining NaNs with zeros.")
        data_filled = np.nan_to_num(data, nan=0)  # Fill any remaining NaNs with zeros
        X_final.append(data_filled)
    else:
        X_final.append(data)

# Ensure consistency in the number of features
if not all(data.shape[1] == X_final[0].shape[1] for data in X_final):
    raise ValueError("All subjects must have the same number of features before standardization.")

# Step 3: Standardize data
pad_X_standardized = [StandardScaler().fit_transform(data) for data in X_final]

# Initialize ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
# Fit and transform to compute connectivity matrices
connectivity_matrices = correlation_measure.fit_transform(pad_X_standardized)
# Use numpy to get the upper triangle of each connectivity matrix
num_subjects = connectivity_matrices.shape[0]
num_nodes = connectivity_matrices.shape[1]

# Get upper triangle indices (excluding diagonal)
upper_tri_indices = np.triu_indices(num_nodes, k=1)

# Flatten and store the upper triangle values in the desired shape
pad_upper_triangle_flattened = np.empty((num_subjects, len(upper_tri_indices[0])))

# Extract upper triangle values for each subject
for i in range(num_subjects):
    pad_upper_triangle_flattened[i] = connectivity_matrices[i][upper_tri_indices]

# Check the shape of the result
print(pad_upper_triangle_flattened.shape) 

sample_size = pad_upper_triangle_flattened.shape[1]
df_PAD = df_participants.loc[df_participants['eid'].isin(subject_id_list_filtered)]
X_pad = pad_upper_triangle_flattened  # Feature matrix
y_pad = y = df_PAD['hospital_not_now']  # Target variable (e.g., symptom scores)


In [301]:
# version 1
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Split data into training and testing sets
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Step 2: Feature selection using RFE with n selected features and 10-fold cross-validation
def feature_selection_with_rfe_cv(X_train, y_train, n, best_model):
    """
    Perform feature selection using Recursive Feature Elimination (RFE) with the best model.
    
    Parameters:
    - X_train: Training feature set
    - y_train: Training labels
    - n: Number of features to select
    - best_model: The model to use as the estimator for RFE

    Returns:
    - selected_features: A boolean mask indicating selected features
    """
    # Ensure the best_model has a fit method
    if not hasattr(best_model, "fit"):
        raise ValueError("The provided best_model must have a fit method.")

    # Initialize RFE with the specified model
    rfe = RFE(estimator=best_model, n_features_to_select=n, step=1)

    # Fit RFE to the training data
    try:
        rfe.fit(X_train, y_train)
    except ValueError as e:
        print(f"Error during RFE fit: {e}")
        raise

    # Get the selected features mask
    selected_features = rfe.support_

    # Ensure at least one feature is selected
    if np.sum(selected_features) == 0:
        print("No features selected. Using all features as fallback.")
        selected_features = np.ones(X_train.shape[1], dtype=bool)

    return selected_features

# Step 3: Model selection using cross-validation
def model_selection(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),
        "Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),
        "Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),
        "ElasticNet (L1+L2)": LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5),
        "LDA": LinearDiscriminantAnalysis(),
        "Perceptron": Perceptron(),
        "SVM (Linear)": SVC(kernel='linear'),
        "Random Forest": RandomForestClassifier()
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    for model_name, model in models.items():
        cv_score = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy').mean()
        print(f"Model: {model_name}, CV Score: {cv_score:.4f}")

        if cv_score > best_score:
            best_score = cv_score
            best_model = model
            best_name = model_name

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model

# Step 4: Two-step grid search for hyperparameter optimization
def tune_model_hyperparameters(best_model, X_train, y_train):
    if isinstance(best_model, SVC):  # Example for SVC
        # Broad grid search
        param_grid = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'kernel': ['linear'],
            'gamma': ['scale', 'auto']
        }
        grid_search = GridSearchCV(best_model, param_grid, scoring='accuracy', cv=10, verbose=1, n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Narrow search around best parameters
        best_params = grid_search.best_params_
        refined_grid = {
            'C': np.linspace(best_params['C'] * 0.1, best_params['C'] * 10, 5),
            'kernel': ['linear'],
            'gamma': ['scale', 'auto']
        }
        refined_search = GridSearchCV(best_model, refined_grid, scoring='accuracy', cv=10, verbose=1, n_jobs=-1)
        refined_search.fit(X_train, y_train)

        print(f"Best SVM Parameters (Refined): {refined_search.best_params_}")
        return refined_search.best_estimator_
    else:
        best_model.fit(X_train, y_train)
        return best_model

# Step 5: Train final model and evaluate with test data
def train_and_evaluate_final_model(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    print(f"Test Set Accuracy (Final Model): {test_accuracy:.4f}")
    return model, test_accuracy

def ensure_binary_target(y):
    """
    Ensure the target variable is binary (0 and 1).
    
    Parameters:
    - y: Target variable (numpy array)

    Returns:
    - y_binary: Binary target variable (0 and 1)
    """
    unique_values = np.unique(y)
    if len(unique_values) > 2:
        raise ValueError("Target variable contains more than two classes. Please preprocess the data.")
    if unique_values.dtype == bool:
        # Convert boolean to integers
        return y.astype(int)
    elif np.array_equal(unique_values, [0, 1]) or np.array_equal(unique_values, [1, 0]):
        # Already binary
        return y
    else:
        raise ValueError("Target variable is not binary. Please preprocess the data.")
    
# Step 6: Calculate cosine similarity between two sets of model weights
def calculate_cosine_similarity(model1, model2):
    """
    Calculate the cosine similarity between two model weights.
    
    Parameters:
    - model1: First trained model
    - model2: Second trained model
    
    Returns:
    - similarity: Cosine similarity score between the two model weights
    """
    # Extract the weights (coefficients) of the models
    if hasattr(model1, 'coef_') and hasattr(model2, 'coef_'):
        weights1 = model1.coef_.flatten()
        weights2 = model2.coef_.flatten()
        similarity = cosine_similarity([weights1], [weights2])
        return similarity[0][0]
    else:
        raise ValueError("Models do not have coefficients. Cosine similarity cannot be computed.")

# Main pipeline with integration for CAD and PAD comparison
def pipeline(X, y, n_features):
    # Ensure target variable is binary
    y = ensure_binary_target(y)

    # Split data
    X_train, X_test, y_train, y_test = split_data(X, y)
    #print(f"After split_data, unique values in y_train: {np.unique(y_train)}")
    #print(f"After split_data, unique values in y_test: {np.unique(y_test)}")

    # Model selection
    best_model = model_selection(X_train, y_train)
    if best_model is None:
        raise ValueError("No valid model was selected. Check your model selection process.")
    
    # Feature selection
    selected_features = feature_selection_with_rfe_cv(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    #print(f"After feature selection, unique values in y_train: {np.unique(y_train)}")

    

    # Hyperparameter tuning
    tuned_model = tune_model_hyperparameters(best_model, X_train_selected, y_train)

    # Train and evaluate final model
    final_model, test_accuracy = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )

    return final_model, selected_features, test_accuracy

# Step 7: Comparison between CAD and PAD models
def compare_models_and_analyze_topography(X_cad, y_cad, X_pad, y_pad, n_features):
    # Train the CAD model and save selected features
    print("Training CAD model...")
    final_model_cad, selected_features_cad, test_accuracy_cad = pipeline(X_cad, y_cad, n_features)

    # Train the PAD model and save selected features
    print("\nTraining PAD model...")
    final_model_pad, selected_features_pad, test_accuracy_pad = pipeline(X_pad, y_pad, n_features)

    # Ensure the selected features from CAD are used in PAD model evaluation
    print("\nEvaluating CAD model on PAD dataset:")
    X_pad_selected = X_pad[:, selected_features_cad]  # Apply CAD-selected features to PAD dataset
    y_pred_pad = final_model_cad.predict(X_pad_selected)
    accuracy = accuracy_score(y_pad, y_pred_pad)
    print(f"Accuracy of CAD model on PAD data: {accuracy:.4f}")

    # Confusion matrix and classification report
    print("Confusion Matrix:\n", confusion_matrix(y_pad, y_pred_pad))
    print("Classification Report:\n", classification_report(y_pad, y_pred_pad))

    # Step 8: Calculate Cosine Similarity between CAD and PAD model weights
    print("\nCalculating cosine similarity between CAD and PAD model weights:")
    similarity = calculate_cosine_similarity(final_model_cad, final_model_pad)
    
    return accuracy, similarity

# Example usage with CAD and PAD datasets
n_features = 30  # Number of features to select
print("Pipeline for CAD dataset:")
final_model_cad, selected_features_cad, test_accuracy_cad = pipeline(X_cad, y_cad, n_features)

print("\nPipeline for PAD dataset:")
final_model_pad, selected_features_pad, test_accuracy_pad = pipeline(X_pad, y_pad, n_features)

print("\nComparing CAD and PAD models:")
accuracy, similarity = compare_models_and_analyze_topography(
    X_cad, y_cad, X_pad, y_pad, n_features
)

print(f"\nCAD model accuracy on PAD data: {accuracy:.4f}")
print(f"Cosine similarity between CAD and PAD model weights: {similarity:.4f}")

Pipeline for CAD dataset:
Model: Logistic Regression, CV Score: 0.5007
Model: Ridge Classifier, CV Score: 0.4992
Model: Lasso (L1), CV Score: 0.5281


/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
 

Model: ElasticNet (L1+L2), CV Score: 0.5108
Model: LDA, CV Score: 0.5367
Model: Perceptron, CV Score: 0.4992
Model: SVM (Linear), CV Score: 0.5005
Model: Random Forest, CV Score: 0.5166
Best Model: LDA with CV score: 0.5367
Test Set Accuracy (Final Model): 0.5057

Pipeline for PAD dataset:
Model: Logistic Regression, CV Score: 0.5276
Model: Ridge Classifier, CV Score: 0.5260
Model: Lasso (L1), CV Score: 0.5211


/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
 

Model: ElasticNet (L1+L2), CV Score: 0.5211
Model: LDA, CV Score: 0.4756
Model: Perceptron, CV Score: 0.4999
Model: SVM (Linear), CV Score: 0.5113
Model: Random Forest, CV Score: 0.5244
Best Model: Logistic Regression with CV score: 0.5276
Test Set Accuracy (Final Model): 0.4710

Comparing CAD and PAD models:
Training CAD model...
Model: Logistic Regression, CV Score: 0.5007
Model: Ridge Classifier, CV Score: 0.4992
Model: Lasso (L1), CV Score: 0.5281


/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
 

Model: ElasticNet (L1+L2), CV Score: 0.5122
Model: LDA, CV Score: 0.5367
Model: Perceptron, CV Score: 0.4992
Model: SVM (Linear), CV Score: 0.5005
Model: Random Forest, CV Score: 0.5165
Best Model: LDA with CV score: 0.5367
Test Set Accuracy (Final Model): 0.5057

Training PAD model...
Model: Logistic Regression, CV Score: 0.5276
Model: Ridge Classifier, CV Score: 0.5260
Model: Lasso (L1), CV Score: 0.5227


/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
 

Model: ElasticNet (L1+L2), CV Score: 0.5227
Model: LDA, CV Score: 0.4756
Model: Perceptron, CV Score: 0.4999
Model: SVM (Linear), CV Score: 0.5113
Model: Random Forest, CV Score: 0.5437
Best Model: Random Forest with CV score: 0.5437
Test Set Accuracy (Final Model): 0.4903

Evaluating CAD model on PAD dataset:
Accuracy of CAD model on PAD data: 0.5110
Confusion Matrix:
 [[182 174]
 [203 212]]
Classification Report:
               precision    recall  f1-score   support

       False       0.47      0.51      0.49       356
        True       0.55      0.51      0.53       415

    accuracy                           0.51       771
   macro avg       0.51      0.51      0.51       771
weighted avg       0.51      0.51      0.51       771


Calculating cosine similarity between CAD and PAD model weights:


ValueError: Models do not have coefficients. Cosine similarity cannot be computed.

In [302]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, Ridge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")


# Step 1: Split data into training and testing sets
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


# Step 2: Preprocess the data (scaling features)
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled


# Step 3: Model selection with cross-validation
def model_selection(X_train, y_train):
    """
    Select the best-performing model based on cross-validation scores.
    """
    models = {
        "Logistic Regression": LogisticRegression(),
        "Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),
        "Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),
        "ElasticNet (L1+L2)": LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5),
        "LDA": LinearDiscriminantAnalysis(),
        "Perceptron": Perceptron(),
        "SVM (Linear)": SVC(kernel='linear', probability=True),
        "Random Forest": RandomForestClassifier(),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    # Loop over each model, compute cross-validation scores, and select the best one
    for model_name, model in models.items():
        try:
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
            mean_score = scores.mean()
            print(f"{model_name}: CV Accuracy = {mean_score:.4f}")
            if mean_score > best_score:
                best_score = mean_score
                best_model = model
                best_name = model_name
        except Exception as e:
            print(f"Skipping model {model_name} due to error: {e}")

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model


# Step 4: Feature selection using RFE with the selected best model
def feature_selection_with_rfe(X_train, y_train, n, model):
    """
    Perform feature selection using Recursive Feature Elimination with the given model.
    """
    try:
        from sklearn.feature_selection import RFE
        rfe = RFE(estimator=model, n_features_to_select=n, step=1)
        rfe.fit(X_train, y_train)
        selected_features = rfe.support_
        print(f"Features selected using RFE: {selected_features}")
        return selected_features
    except Exception as e:
        print(f"Error during RFE with the selected model: {e}")
        # Fallback to all features
        return np.ones(X_train.shape[1], dtype=bool)


# Step 5: Train ensemble model with selected features
def train_ensemble_model(X_train, y_train):
    """
    Create and train a voting ensemble of classifiers.
    """
    ensemble = VotingClassifier(
        estimators=[
            ('lr', LogisticRegression(max_iter=1000)),
            ('svm', SVC(kernel='linear', probability=True)),
            ('rf', RandomForestClassifier()),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
        ],
        voting='soft',
        n_jobs=-1
    )
    ensemble.fit(X_train, y_train)
    return ensemble


# Step 6: Train and evaluate the final model on test data
def train_and_evaluate_model(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"Test Accuracy: {test_accuracy:.4f}")
    # Compute ROC-AUC if possible
    roc_auc = None
    if hasattr(model, "predict_proba"):
        roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        print(f"Test ROC-AUC: {roc_auc:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, model.predict(X_test)))
    print("Classification Report:\n", classification_report(y_test, model.predict(X_test)))
    return test_accuracy, roc_auc


# Main pipeline
def pipeline(X, y, n_features=20):
    # Ensure target variable is binary
    y = np.array(y)
    if len(np.unique(y)) != 2:
        raise ValueError("Target variable must be binary.")

    # Split the data
    X_train, X_test, y_train, y_test = split_data(X, y)

    # Preprocess data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Model selection
    best_model = model_selection(X_train, y_train)

    # Perform feature selection using RFE
    selected_features_mask = feature_selection_with_rfe(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features_mask]
    X_test_selected = X_test[:, selected_features_mask]

    # Train ensemble model
    ensemble_model = train_ensemble_model(X_train_selected, y_train)

    # Evaluate the ensemble model
    accuracy, roc_auc = train_and_evaluate_model(X_train_selected, y_train, X_test_selected, y_test, ensemble_model)

    return ensemble_model, accuracy, roc_auc


# Example usage with CAD and PAD datasets
# Ensure X_cad, y_cad, X_pad, and y_pad are defined
n_features = 30
pipeline(X_cad, y_cad, n_features)

Logistic Regression: CV Accuracy = 0.5008
Ridge Classifier: CV Accuracy = 0.4993
Lasso (L1): CV Accuracy = 0.4878
ElasticNet (L1+L2): CV Accuracy = 0.4763
LDA: CV Accuracy = 0.5267
Perceptron: CV Accuracy = 0.5181
SVM (Linear): CV Accuracy = 0.5238
Random Forest: CV Accuracy = 0.5325
XGBoost: CV Accuracy = 0.5079
Best Model: Random Forest with CV score: 0.5325
Features selected using RFE: [False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False  True False False False False False False
 False False False False

/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:14:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [18:14:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.5632
Test ROC-AUC: 0.5810
Confusion Matrix:
 [[41 34]
 [42 57]]
Classification Report:
               precision    recall  f1-score   support

       False       0.49      0.55      0.52        75
        True       0.63      0.58      0.60        99

    accuracy                           0.56       174
   macro avg       0.56      0.56      0.56       174
weighted avg       0.57      0.56      0.57       174



(VotingClassifier(estimators=[('lr', LogisticRegression(max_iter=1000)),
                              ('svm', SVC(kernel='linear', probability=True)),
                              ('rf', RandomForestClassifier()),
                              ('xgb',
                               XGBClassifier(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_m...
                                             importance_type=None,
                                             interaction_constraints=None,
                                             learn